In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(color_codes=True) #overide maplot libs ugly colours.
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [14, 8] #default figure size
import math
%matplotlib inline

In [3]:
df = pd.read_csv('data\processed_data(by_quarter).csv')

In [4]:
df.drop(['Unnamed: 0', "usd_pledged"], axis=1, inplace=True)

In [5]:
df.head()

,duration,goal_usd,blurb_length,name_length,success,US based,main_category_comics,main_category_crafts,main_category_dance,main_category_design,...,main_category_games,main_category_journalism,main_category_music,main_category_photography,main_category_publishing,main_category_technology,main_category_theater,start_Q_Q2,start_Q_Q3,start_Q_Q4
0,0.163043,0.000015,0.382353,0.230769,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.315217,0.000030,0.676471,0.269231,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.315217,0.000009,0.588235,0.230769,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.478261,0.000027,0.411765,0.192308,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.641304,0.000232,0.411765,0.115385,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [23]:
def random_forest_test(df, y_column,  depth):
    
    import pandas as pd

    # y_column is a string for the column name
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier

    # define X and y from the dataframe
    y = df[y_column]
    X = df.drop(labels = y_column, axis = 'columns')

    # making the training set and the testing set
    from sklearn.model_selection import train_test_split
    X_train, X_test ,y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Instantiate the estimator
    clf = RandomForestClassifier(max_depth = depth, random_state = 42, n_jobs=-1)

    # Training the model
    clf.fit(X_train, y_train)

    # Making a prediction
    y_pred = clf.predict(X_test)

    # Checking the accuracy
    from sklearn.metrics import accuracy_score
    a_score = accuracy_score(y_test, y_pred)

    # Calculating probability of success
    probability = clf.predict_proba(X_test)

    # Calculate the confusion matrix
    from sklearn.metrics import confusion_matrix
    c_matrix = confusion_matrix(y_test, y_pred)

    # Calculate the classification
    from sklearn.metrics import classification_report
    c_report = classification_report(y_test, y_pred)
    
    #11: KFolds cross validation
    from sklearn.model_selection import cross_val_score
    accuracy = cross_val_score(clf, X_train, y_train, cv = 5)
    print ('KFolds cross validation: \n ', accuracy)
    print ()
    print ('Mean accuracy: \n', accuracy.mean())
    print ()
    
    
    print("Feature Importance: \n", )
    for index, i in enumerate(clf.feature_importances_):
        print (f'Feature {index}: {i:.5f}')
    print()
    print("Model: \n",clf)
    print()
    print("y prediction: \n", y_pred)
    print()
    print("Accuracy score: \n", a_score)
    print()
    print("Probability: \n", probability)
    print()
    print("Confusion matrix: \n", c_matrix)
    print()
    print("Classification report: \n", c_report)
    

In [24]:
random_forest_test(df, "success", 5)

KFolds cross validation: 
  [0.66826518 0.66925358 0.66918036 0.66775268 0.66195636]

Mean accuracy: 
 0.6672816338277452

Feature Importance: 

Feature 0: 0.12985
Feature 1: 0.34838
Feature 2: 0.00772
Feature 3: 0.11295
Feature 4: 0.00628
Feature 5: 0.03589
Feature 6: 0.00767
Feature 7: 0.01080
Feature 8: 0.00123
Feature 9: 0.00003
Feature 10: 0.00315
Feature 11: 0.14477
Feature 12: 0.00197
Feature 13: 0.01912
Feature 14: 0.02172
Feature 15: 0.00756
Feature 16: 0.03069
Feature 17: 0.10840
Feature 18: 0.00033
Feature 19: 0.00036
Feature 20: 0.00070
Feature 21: 0.00044

Model: 
 RandomForestClassifier(max_depth=5, n_jobs=-1, random_state=42)

y prediction: 
 [1. 1. 0. ... 1. 1. 1.]

Accuracy score: 
 0.6700931295027236

Probability: 
 [[0.34167154 0.65832846]
 [0.32841109 0.67158891]
 [0.5070699  0.4929301 ]
 ...
 [0.36933648 0.63066352]
 [0.3440063  0.6559937 ]
 [0.31819326 0.68180674]]

Confusion matrix: 
 [[ 6313  8589]
 [ 2676 16568]]

Classification report: 
               precisio

## Optimizing Hyperparameters

In [10]:
def tuning_forest(df, y_column):
    
    import pandas as pd

    # y_column is a string for the column name
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier

    # define X and y from the dataframe
    y = df[y_column]
    X = df.drop(labels = y_column, axis = 'columns')

    # making the training set and the testing set
    from sklearn.model_selection import train_test_split
    X_train, X_test ,y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Instantiate the estimator
    clf = RandomForestClassifier()

    # Training the model
    clf.fit(X_train, y_train)


    parameters = {'max_depth': list(range(1,5))}

    from sklearn.model_selection import GridSearchCV

    search = GridSearchCV(clf, param_grid= parameters, cv=10)
    search.fit(X_train,y_train)

    print(search.best_score_)
    print(search.best_params_)

In [11]:
tuning_forest(df, 'success')

0.6616514164950574
{'max_depth': 4}


In [25]:
random_forest_test(df, 'success',  4)

KFolds cross validation: 
  [0.66284731 0.66672768 0.66401874 0.66284731 0.65946698]

Mean accuracy: 
 0.6631816046196805

Feature Importance: 

Feature 0: 0.12609
Feature 1: 0.33466
Feature 2: 0.00536
Feature 3: 0.11160
Feature 4: 0.00449
Feature 5: 0.03607
Feature 6: 0.00517
Feature 7: 0.01122
Feature 8: 0.00122
Feature 9: 0.00004
Feature 10: 0.00165
Feature 11: 0.15445
Feature 12: 0.00207
Feature 13: 0.02478
Feature 14: 0.02850
Feature 15: 0.00669
Feature 16: 0.03339
Feature 17: 0.11135
Feature 18: 0.00030
Feature 19: 0.00017
Feature 20: 0.00054
Feature 21: 0.00018

Model: 
 RandomForestClassifier(max_depth=4, n_jobs=-1, random_state=42)

y prediction: 
 [1. 1. 0. ... 1. 1. 1.]

Accuracy score: 
 0.6650559362736485

Probability: 
 [[0.34628281 0.65371719]
 [0.34083685 0.65916315]
 [0.51447446 0.48552554]
 ...
 [0.37174012 0.62825988]
 [0.35996568 0.64003432]
 [0.33114083 0.66885917]]

Confusion matrix: 
 [[ 5897  9005]
 [ 2432 16812]]

Classification report: 
               precisio